In [3]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/dlfs-utils/��Դ���롿���ѧϰ���ţ�����Python��������ʵ��/LICENSE.md
/kaggle/input/dlfs-utils/��Դ���롿���ѧϰ���ţ�����Python��������ʵ��/.gitignore
/kaggle/input/dlfs-utils/��Դ���롿���ѧϰ���ţ�����Python��������ʵ��/README.md
/kaggle/input/dlfs-utils/��Դ���롿���ѧϰ���ţ�����Python��������ʵ��/ch02/nand_gate.py
/kaggle/input/dlfs-utils/��Դ���롿���ѧϰ���ţ�����Python��������ʵ��/ch02/xor_gate.py
/kaggle/input/dlfs-utils/��Դ���롿���ѧϰ���ţ�����Python��������ʵ��/ch02/or_gate.py
/kaggle/input/dlfs-utils/��Դ���롿���ѧϰ���ţ�����Python��������ʵ��/ch02/and_gate.py
/kaggle/input/dlfs-utils/��Դ���롿���ѧϰ���ţ�����Python��������ʵ��/ch08/train_deepnet.py
/kaggle/input/dlfs-utils/��Դ���롿���ѧϰ���ţ�����Python��������ʵ��/ch08/deep_convnet_params.pkl
/kaggle/input/dlfs-utils/��Դ���롿���ѧϰ���ţ�����Python��������ʵ��/ch08/misclassified_mnist.py
/kaggle/input/dlfs-utils/��Դ���롿���ѧϰ���ţ�����Python��������ʵ��/ch08/awesome_net.py
/kaggle/input/dlfs-utils/��Դ���롿���ѧϰ���ţ�����Python��������ʵ��/ch08/half_float_network.py
/kaggl

In [4]:
!git clone https://github.com/oreilly-japan/deep-learning-from-scratch.git

fatal: destination path 'deep-learning-from-scratch' already exists and is not an empty directory.


In [5]:
%cd deep-learning-from-scratch/

/kaggle/working/deep-learning-from-scratch


In [6]:
def softmax(x):
    x = x - np.max(x, axis=-1, keepdims=True)
    return np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True)


In [7]:
import sys, os
sys.path.append(os.pardir)
from dataset.mnist import load_mnist


In [8]:
import pickle


def softmax(x):
    x = x - np.max(x, axis=-1, keepdims=True)
    return np.exp(x) / np.sum(np.exp(x), axis=-1, keepdims=True)


def get_data():
    
    (x_train, t_train), (x_test, t_test) = load_mnist(flatten=True, normalize=False, one_hot_label=False)
    return x_test, t_test
    
def init_network():
    with open("ch03/sample_weight.pkl", 'rb') as f:
        network = pickle.load(f)
        return network
    
def predict(network, x):
    w1, w2, w3 = network['W1'], network['W2'], network['W3']
    b1, b2, b3 = network['b1'], network['b2'], network['b3']
    
    a1 = np.dot(x, w1)+b1
    z1 = sigmoid(a1)
    
    a2 = np.dot(z1, w2) + b2
    z2 = sigmoid(a2)
    
    a3 = np.dot(z2, w3) + b3
    y = softmax(a3)
    return y

In [9]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
    batch_size = y.shape[0]
    return -np.sum(t*np.log(y + 1e-7)) / batch_size

In [10]:
class MulLayer:
    def __init__(self):
        self.x = None
        self.y = None
        
    def forward(self, x, y):
        self.x = x
        self.y = y
        out = x * y
        return out
    
    def backward(self, dout):
        dx = dout * self.y
        dy = dout * self.x
        return dx, dy
    

In [11]:
class AddLayer:
    def __init__(self):
        pass
        
    def forward(self, x, y):
        return x + y
    
    def backward(self, dout):
        return dout, dout

In [12]:
class Relu:
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out
        
    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

In [13]:
class Sigmoid:
    def __init__(self):
        self.x = None
        
    def forward(self, x):
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out
    
    def backward(self, dout):
        dx = dout * (1.0 - self.out) * self.out
        return dx

In [14]:
class Affine:
    def __init__(self, W, b):
        self.W = W
        self.b = b
        self.x = None
        self.dW = None
        self.db = None
        
    def forward(self, x):
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

In [15]:
class SoftmaxWithLoss:
    def __init__(self):
        self.loss = None
        self.y = None
        self.t = None
        
    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

In [16]:

class TwoLayerNet:
    def __init__(self, input_size, hidden_size, output_size,
                weight_init_std=0.01):
        self.params = {}
        self.params['W1'] = weight_init_std * np.random.randn(input_size, hidden_size)
        self.params['b1'] = np.zeros(hidden_size)
        self.params['W2'] = weight_init_std * np.random.randn(hidden_size, output_size)
        self.params['b2'] = np.zeros(output_size)
        
        self.layers = dict()
        self.layers['Affine1'] = Affine(self.params['W1'], self.params['b1'])
        self.layers['Relu1'] = Relu()
        self.layers['Affine2'] = Affine(self.params['W2'], self.params['b2'])
        
        self.lastlayer = SoftmaxWithLoss()
        
    def predict(self, x):
        for layer in self.layers.values():
            x = layer.forward(x)

            
        return x
    
    
    def loss(self, x, t):
        y = self.predict(x)
        return self.lastlayer.forward(y, t)
    
    
    def accuracy(self, x, t):
        y = self.predict(x)
        y = np.argmax(y, axis=1)
        if t.ndim!=1:
            t = np.argmax(t, axis=1)
        accuracy = np.sum(y==t) / float(x.shape[0])
        return accuracy
    
    def gradient(self, x, t):
        self.loss(x, t)

        
        dout = 1
        dout = self.lastlayer.backward(dout)
        
        layers = list(self.layers.values())
        layers.reverse()
        
        for layer in layers:
            # print(f'lllllllllayer: {layer}, {dout}')
            dout = layer.backward(dout)

            
        grads = {}
        grads['W1'] = self.layers['Affine1'].dW
        grads['b1'] = self.layers['Affine1'].db
        grads['W2'] = self.layers['Affine2'].dW
        grads['b2'] = self.layers['Affine2'].db
        return grads

In [21]:
(x_train, t_train), (x_test, t_test) = load_mnist(normalize=True, one_hot_label=True)
network = TwoLayerNet(input_size=784, hidden_size=50, output_size=10)

iters_num = 10000
train_size = x_train.shape[0]
batch_size = 100
learning_rate = 0.09
train_loss_list = []
train_acc_list = []
test_acc_list = []

iter_per_epoch = max(train_size / batch_size, 1)

for i in range(iters_num):
    batch_mask = np.random.choice(train_size, batch_size)
    x_batch = x_train[batch_mask]
    t_batch = t_train[batch_mask]
    grad = network.gradient(x_batch, t_batch)
    
    for key in ['W1', 'b1', 'W2', 'b2']:
        network.params[key] -= learning_rate * grad[key]
    
    loss = network.loss(x_batch, t_batch)
    train_loss_list.append(loss)
    
    
    if i % iter_per_epoch == 0:
        train_acc = network.accuracy(x_train, t_train)
        test_acc = network.accuracy(x_test, t_test)
        train_acc_list.append(train_acc)
        test_acc_list.append(test_acc)
        print(train_acc, test_acc)

0.11145 0.1204
0.9004666666666666 0.9029
0.92165 0.925
0.931 0.9289
0.94125 0.9391
0.9488666666666666 0.9451
0.95475 0.9529
0.95785 0.9533
0.96105 0.9559
0.9643166666666667 0.9577
0.96855 0.9608
0.9692833333333334 0.9622
0.9723166666666667 0.9642
0.9739166666666667 0.9661
0.9755666666666667 0.9672
0.97675 0.9683
0.9785166666666667 0.9693


In [25]:
class SGD:
    def __init__(self, lr=0.01):
        self.lr = lr
        
    def update(self, params, grads):
        for key in params.keys():
            params[key] -= self.lr * grads[key]

In [ ]:
class Momentum:
    def __init__(self, lr=0.01, momentum=0.9):
        self.lr = lr
        self.mementum = momentum
        self.v = None
    
    def update(self, params, grads):
        if self.v is None:
            self.v = {}
            for k, v in params.items():
                self.v[k] = np.zeros_like(val)
        for key in params.keys():
            self.v[key] = self.mementum*self.v[key] - grads[k] * self.lr
            params[key] += self.v[key]